In [1]:
# global import
import os
import numpy as np
import utils
import pickle

# global variable
voxel_size = 0.1 # which means 0.1 meter


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
if os.path.exists("./data/checkpoint/voxels.pkl") and os.path.exists("./data/checkpoint/colors.pkl"):
    with open("./data/checkpoint/voxels.pkl", "rb") as f:
        voxels = pickle.load(f)
    with open("./data/checkpoint/colors.pkl", "rb") as f:
        colors = pickle.load(f)
else:
    raw_points, raw_colors = utils.las2npy("../../dataset/001-010.las") # shape (169438332, 3)
    # rarefaction 体素滤波
    raw_points = raw_points[::50]
    raw_colors = raw_colors[::50]
    voxels, idx_o2n, idx_n2o, unique_counts = utils.voxel_downsample(raw_points, voxel_size, True)
    colors = raw_colors[idx_o2n]
    with open("./data/checkpoint/voxels.pkl", "wb") as f:
        pickle.dump(voxels, f)
    with open("./data/checkpoint/colors.pkl", "wb") as f:
        pickle.dump(colors, f)

utils.npy2ply(voxels, colors, "./data/pipeline_original.ply")
print(f"number of points after downsampled: {voxels.shape}")


number of points after downsampled: (3352104, 3)


# Height filtration section

In [3]:
# 执行基于高程信息的基础滤波，这种滤波方法不能直接对整个点云集合处理，
# 因为输电线路的架设不一定维持在同一海平面，均值滤波需要分成小批次处理
start_point = 0
march_steop = int(2.5e3)
pivot_list = [] # pivot list
stapt_list = [] # start point list

boundary = voxels.shape[0]
while start_point < boundary:
    pivot = utils.get_average_pivot(voxels[start_point:min(start_point + march_steop, boundary), 2])
    pivot_list.append(pivot)
    stapt_list.append(start_point)
    start_point += march_steop

height_filter_mask = np.zeros((0, ), dtype=bool)
for stapt, pivot in zip(stapt_list, pivot_list):
    height_filter_mask = np.concatenate((height_filter_mask, voxels[stapt:min(stapt + march_steop, boundary), 2] > pivot))

voxels_filteredby_height = voxels[height_filter_mask]
colors_filteredby_height = colors[height_filter_mask]

utils.npy2ply(voxels_filteredby_height, colors_filteredby_height, "./data/pipeline_height_filtration.ply", use_txt=False)


# Power line extraction

In [4]:
# 多进程处理子空间坐标几何特征计算
from concurrent.futures import ProcessPoolExecutor, as_completed

if os.path.exists("./data/checkpoint/line_feat_mat.pkl"):
    with open("./data/checkpoint/line_feat_mat.pkl", "rb") as f:
        line_feat_mat = pickle.load(f)
else:
    num_workers = 4
    batch_size = voxels_filteredby_height.shape[0] // num_workers
    line_feat_mat_list = [None] * num_workers
    with ProcessPoolExecutor(max_workers=num_workers) as executor:
        futures_dict = {
            executor.submit(
                utils.eigval_radius,
                voxels_filteredby_height[i * batch_size : (i + 1) * batch_size],
                voxel_size * 75.0
            ) : i for i in range(num_workers)
        }
        for future in as_completed(futures_dict):
            feat_mat, _ = future.result()
            line_feat_mat_list[futures_dict[future]] = feat_mat
    line_feat_mat = np.concatenate(line_feat_mat_list, axis=0)
    with open("./data/checkpoint/line_feat_mat.pkl", "wb") as f:
        pickle.dump(line_feat_mat, f)


In [5]:
# 因为使用了多进程分组处理，所以有可能原先分组不能整除刚好分给各个进程
# 因此处理后的掩码长度也许和原点云坐标数量不一，需要按照掩码的长度调整
voxels_filteredby_height = voxels_filteredby_height[:line_feat_mat.shape[0]]
colors_filteredby_height = colors_filteredby_height[:line_feat_mat.shape[0]]

line_feat_max = np.max(line_feat_mat, axis=0)
line_feat_min = np.min(line_feat_mat, axis=0)
print(line_feat_max)
print(line_feat_min)
line_feat_rgb = ((line_feat_mat - line_feat_min) / line_feat_max * 255.0).astype(np.int32)
print(line_feat_rgb)
utils.npy2ply(voxels_filteredby_height, line_feat_rgb, "./data/pipeline_height_filtration_vis.ply")


[51.03776573 51.10588386 14.84088054]
[0. 0. 0.]
[[  0   0   0]
 [  7   9  23]
 [  9  27  51]
 ...
 [ 15  51  55]
 [  9  55  98]
 [  8  65 133]]


In [6]:
line_mask = (line_feat_mat[:, 0] > line_feat_mat[:, 0].mean()) & (line_feat_mat[:, 1] > line_feat_mat[:, 1].mean())
# 获取电力线点云
voxels_line = voxels_filteredby_height[line_mask]
colors_line = colors_filteredby_height[line_mask]

def radius_filtration(data: np.ndarray, radius: float):
    import open3d as o3d
    from concurrent.futures import ThreadPoolExecutor, as_completed

    o3d_pcd = utils.npy2o3d(data)
    search_tree = o3d.geometry.KDTreeFlann(o3d_pcd)

    def radius_search(idx):
        k, idx, _ = search_tree.search_radius_vector_3d(data[idx], radius)
        return k

    num_neigbgours = np.array([0] * len(data))
    with ThreadPoolExecutor() as executor:
        futures_dict = {executor.submit(radius_search, i) : i for i in range(len(data))}
        for future in as_completed(futures_dict):
            num_neigbgours[futures_dict[future]] = future.result()
    
    return num_neigbgours > num_neigbgours.mean() / 4.0

# line_noise_filter_mask = radius_filtration(voxels_line, voxel_size * 30.0)
# voxels_line = voxels_line[line_noise_filter_mask]
# colors_line = colors_line[line_noise_filter_mask]

utils.npy2ply(voxels_line, colors_line, "./data/pipeline_line.ply")


# Tower extraction

In [7]:
line_excluded_mask = ~line_mask & (line_feat_mat[:, 0] < line_feat_mat[:, 0].mean()) & (line_feat_mat[:, 1] < line_feat_mat[:, 1].mean())
voxels_line_excluded = voxels_filteredby_height[line_excluded_mask]
colors_line_excluded = colors_filteredby_height[line_excluded_mask]
utils.npy2ply(voxels_line_excluded, colors_line_excluded, "./data/pipeline_line_excluded.ply")


In [8]:
if os.path.exists("./data/checkpoint/tower_feat_mat.pkl"):
    with open("./data/checkpoint/tower_feat_mat.pkl", "rb") as f:
        tower_feat_mat = pickle.load(f)
else:
    num_workers = 4
    batch_size = voxels_line_excluded.shape[0] // num_workers
    tower_feat_mat_list = [None] * num_workers
    from concurrent.futures import ProcessPoolExecutor, as_completed
    with ProcessPoolExecutor(max_workers=num_workers) as executor:
        futures_dict = {
            executor.submit(
                utils.eigval_vertic,
                voxels_line_excluded[i * batch_size:(i + 1) * batch_size],
                voxel_size * 75.0
            ) : i for i in range(num_workers)
        }

        for future in as_completed(futures_dict):
            feat_mat, _ = future.result()
            tower_feat_mat_list[futures_dict[future]] = feat_mat
    tower_feat_mat = np.concatenate(tower_feat_mat_list, axis=0)
    with open("./data/checkpoint/tower_feat_mat.pkl", "wb") as f:
        pickle.dump(tower_feat_mat, f)


In [9]:
# 也许这里会损失一些额外的点云，由于前面的处理方法的遗留问题
voxels_line_excluded = voxels_line_excluded[:len(tower_feat_mat)]
colors_line_excluded = colors_line_excluded[:len(tower_feat_mat)]

tower_feat_max = np.max(tower_feat_mat, axis=0)
tower_feat_min = np.min(tower_feat_mat, axis=0)
tower_feat_rgb = ((tower_feat_mat - tower_feat_min) / tower_feat_max * 255.0).astype(np.int32)
utils.npy2ply(voxels_line_excluded, tower_feat_rgb, "./data/pipeline_line_excluded_vis.ply")

tower_feat_mat = (tower_feat_mat - tower_feat_min) / tower_feat_max # normalized to [0, 1]
half_max_height = np.median(tower_feat_mat[:, 1])
tower_mask = \
    (tower_feat_mat[:, 0] > 0.25) & \
    (tower_feat_mat[:, 1] > half_max_height) & \
    (tower_feat_mat[:, 2] < 0.1)
print(tower_mask.astype(np.int32).sum())
voxels_tower = voxels_line_excluded[tower_mask]
colors_tower = colors_line_excluded[tower_mask]
utils.npy2ply(voxels_tower, colors_tower, "./data/pipeline_tower.ply")


79207


In [10]:
# denoised 降噪
labels = np.array(utils.npy2o3d(voxels_tower).cluster_dbscan(eps=voxel_size * 75.0, min_points=100, print_progress=False))
if labels.shape[0] != voxels_tower.shape[0]:
    raise ValueError("mismatched label num")

print(f"number of clustered towers: {labels.max() + 1}")
unique_labels, labels_count = np.unique(labels, return_counts=True)

unique_labels = unique_labels[1:]
labels_count = labels_count[1:]
min_threshold = labels_count.mean() // 2
label_selection = labels_count > min_threshold
valid_labels = unique_labels[label_selection]
print(f"tower point threshold: {min_threshold}")

print(f"valid labels: {valid_labels}")
tower_denoised_mask = [(labels == label_idx) for label_idx in valid_labels]
tower_denoised_mask = np.logical_or.reduce(
    np.vstack(tower_denoised_mask),
    axis=0
)
voxels_tower_denoised = voxels_tower[tower_denoised_mask]
colors_tower_denoised = colors_tower[tower_denoised_mask]

utils.npy2ply(
        voxels_tower_denoised,
        colors_tower_denoised,
    "./data/pipeline_tower_denoised.ply"
)


number of clustered towers: 11
tower point threshold: 3588.0
valid labels: [ 0  1  2  3  5  6  7  8  9 10]


In [11]:
tower_instanced_labels = utils.cluster_instanced(voxels_tower_denoised, voxel_size * 200.0, min_threshold)
# 自己写的这个基于垂直方向的聚类函数，标签是从1开始的，因此不需要在最大标签值+1，因为这里的标签下标不是从0开始的
print(f"number of cluster: {tower_instanced_labels.max()}")
voxels_tower_instanced = voxels_tower_denoised
colors_tower_instanced = np.vstack([np.array([0, 0, 0])] * len(voxels_tower_instanced))
for label in range(1, tower_instanced_labels.max() + 1):
    print(f"tower#{label}: {(tower_instanced_labels == label).astype(np.int32).sum():05d}")
    colors_tower_instanced[tower_instanced_labels == label] = np.array([
        np.random.randint(100, 255),
        np.random.randint(100, 255),
        np.random.randint(100, 255)
    ])
utils.npy2ply(voxels_tower_instanced, colors_tower_instanced, "./data/pipeline_tower_instanced.ply")


number of cluster: 10
tower#1: 07304
tower#2: 15903
tower#3: 05230
tower#4: 08318
tower#5: 08530
tower#6: 05297
tower#7: 08097
tower#8: 07507
tower#9: 07592
tower#10: 03610


# Segmentation using line's and tower's position

In [12]:
print(f"number of voxels in line:\t{len(voxels_line)}")
print(f"number of voxels in tower:\t{len(voxels_tower)}")

def position_instanced_radius(points: np.ndarray, poses: np.ndarray, radius: float):
    import open3d as o3d
    
    o3d_pcd = utils.npy2o3d(points)
    search_tree = o3d.geometry.KDTreeFlann(o3d_pcd)

    instanced_labels = np.zeros((len(points), ), dtype=bool)

    for query in poses:
        neighbour_num, neighbour_indicies, _ = search_tree.search_radius_vector_3d(query, radius)
        instanced_labels[neighbour_indicies] = True
    
    return instanced_labels

def position_instanced_vertical(points: np.ndarray, poses: np.ndarray, border: float):
    instanced_labels = np.zeros((len(points), ), dtype=np.int32)

    instanced_labels = np.zeros((len(points), ), dtype=bool)
    for query in poses:
        square_selection = (np.abs(points[:, 0] - query[0]) < border) & (np.abs(points[:, 1] - query[1]) < border)
        instanced_labels = instanced_labels | square_selection
    
    return square_selection



labels_line = position_instanced_radius(voxels, voxels_line, voxel_size * 20.0)
labels_tower = np.array([False] * len(voxels))

for tower_idx in range(1, tower_instanced_labels.max() + 1):
    labels_tower = labels_tower | position_instanced_vertical(
        voxels,
        np.array([voxels_tower_denoised[tower_instanced_labels == tower_idx].mean(axis=0)]),
        voxel_size * 125.0
    )

print(f"number of label in line:\t{labels_line.astype(np.int32).sum()}")
print(f"number of label in tower:\t{labels_tower.astype(np.int32).sum()}")


# give different colors to line and tower
colors_labeled = colors
colors_labeled[labels_line] = np.array([100, 255, 255])
colors_labeled[labels_tower] = np.array([255, 100, 100])

utils.npy2ply(voxels, colors_labeled, "./data/pipeline_final.ply")


number of voxels in line:	86354
number of voxels in tower:	79207
number of label in line:	138810
number of label in tower:	241311


# Line Segmentation

In [13]:
# 根据电力线提取主要走向
_, main_axis = utils.pca_k(voxels_line, 1)
tower_indicies = [i for i in range(1, tower_instanced_labels.max() + 1)]
tower_centers = [voxels_tower_denoised[tower_instanced_labels == tower_idx].mean(axis=0) for tower_idx in tower_indicies]

# 求杆塔中心到走线主方向上的投影长度，并以此为排序依据
# 1. 假设有向量u, v
# 2. u 投影到 v 上得到 u'
# 3. u' 的模长是d
# 4. v / |v| 得到主方向的单位向量
# 5. d * (v / |v|) 得到 u 投影到 v 上的实际最终向量
# 6. d = |u| * cos(theta)
# 7. cose(theta) = (u * v) / (|u| * |v|)
# 8. u' = (u * v) / (|v|^2) * v 
tower_centers_on_main_axis = [
    np.linalg.norm((np.dot(tower_center, main_axis) / np.linalg.norm(main_axis) ** 2) * main_axis) for tower_center in tower_centers
]

# 根据在主方向上投影的距离来排序杆塔
tower_sequence = np.array(tower_centers_on_main_axis).argsort()[::-1]

# 选取相邻的杆塔来截取电力线
line_steps_mask = []
for idx in range(len(tower_sequence) - 1):
    tower_center1 = tower_centers[tower_sequence[idx]]
    tower_center2 = tower_centers[tower_sequence[idx + 1]]

    # 求得垂直于线段的两个平面，两个平面之间的电力线点云分为一档电力线
    
    normal_vec = tower_center1 - tower_center2
    normal_vec[2] = 0.0
    d1 = -np.dot(normal_vec, tower_center1)
    d2 = -np.dot(normal_vec, tower_center2)

    # 如果距离两个平面的距离之乘积是负数，说明该点位于两个平面之间
    line_steps_mask.append(
        (np.dot(voxels[labels_line], normal_vec) + d1) * \
        (np.dot(voxels[labels_line], normal_vec) + d2) < 0.0
    )

# for mask in line_steps_mask:
#     print(f"{mask.astype(np.int32).sum()} / {mask.shape[0]}")

colors_line_seperated = colors
for mask in line_steps_mask:
    colors_line_seperated[np.where(labels_line)[0][mask]]= np.array([
        np.random.randint(50, 255),
        np.random.randint(50, 255),
        np.random.randint(50, 255)
    ])

utils.npy2ply(
    voxels,
    colors_line_seperated,
    "./data/pipeline_powerline_step.ply"
)


## Line Instantiation

In [50]:
# grab a sample to experiment

# the variable `labels_line` is the mask of power line among the
# original point cloud. Then `line_steps_mask` further  seperate
# power line points into several segments between each tower.
line_sample = voxels[np.where(labels_line)[0][line_steps_mask[8]]]

eigvals, eigvecs = utils.pca_k(line_sample, 3)
# transform to new basis given by PCA
line_sample = line_sample @ eigvecs

color_sample = np.array([[100, 100, 100]] * line_sample.shape[0])
print(color_sample.shape)
for i in range(3):
    line_sample_view = np.copy(line_sample)
    line_sample_view[:, i] = 0.0
    utils.npy2ply(line_sample_view, color_sample, f"./data/test_sample_axis{i}.ply")

# projected_to_axis1 = line_sample @ eigvecs[:, 0]
# min_on_axis1 = np.min(projected_to_axis1)
# max_on_axis1 = np.max(projected_to_axis1)
# num_slices = 100 # hyperparameter
# len_per_slice = (max_on_axis1 - min_on_axis1) / num_slices
# projected_to_axis1 = projected_to_axis1 - min_on_axis1
# transformed_cnt = []
# for i in range(num_slices):
#     mask = (projected_to_axis1 > (i * len_per_slice)) & (projected_to_axis1 < ((i + 1) * len_per_slice))
#     line_segment = line_sample[mask][:, 1:] # noneed to count in the first axis
#     if line_segment.shape[0] > 8:
#         center_point = np.mean(line_segment, axis=0)
#         transformed_cnt.append(line_segment - center_point)
# transformed_cnt = np.vstack(transformed_cnt)


# # do some visualization
# import matplotlib.pyplot as plt
# import seaborn as sns
# sns.set_style("darkgrid")

# plot_data_x = transformed_cnt[:, 0]
# plot_data_y = transformed_cnt[:, 1]
# sns.scatterplot(x=plot_data_x, y=plot_data_y)


(4416, 3)
